In [1]:
import sys
sys.path.append('/home/harish/GitRepos/Smart_Helm/')
import os
from typing import List
from minio import Minio
from collections import defaultdict
import json
from pathlib import *
from typing import Dict, List
from pathlib import Path
from utilities.data_downloader import download

In [2]:
bucket_name = "smarthelm-data-accumulation-csl"
#bucket_name = "my-bucket"
folder_path = "CSL"

In [3]:
smarthelm_client = Minio(
        "smarthelm-nas.vlba.uni-oldenburg.de:9000",
        access_key=os.getenv("MINIO_ACCESS_KEY"),
        secret_key=os.getenv("MINIO_SECRET_KEY"),
        secure=False,
)

In [4]:
def scrap(folder_path: str, bucket_name: str) -> List[str]:
    objects = smarthelm_client.list_objects(
        bucket_name=bucket_name,
        prefix=folder_path,
        recursive=True
    )
    filepaths = [obj.object_name for obj in objects if obj.size > 10]
    filtered_filepaths = [path for path in filepaths]
#     for path in filepaths:
#         if not any([ele in path for ele in EXCLUDED_FOLDERS]):
#             filtered_filepaths.append(path)
    return filtered_filepaths

In [5]:
file_paths = scrap(folder_path, bucket_name)

In [7]:
keys_list = ["raw_eeg", "vision" , "location"]
# file_paths[0:6]

In [55]:
d= {}
for i in file_paths:
    if x in keys_list == [y for y in i.split("/")] :
        parti_str = i.partition(x)
        d[parti_str[0]] = {x: i}
        #d = {parti_str[0]: {x: i}}


In [13]:
breakers = ['vision', 'raw_eeg', 'attention', 'location']
j = []
for i in file_paths:
    if  i.split('/')[6] in breakers:
        j.append(list(i.partition(i.split('/')[6])))
    #d = {ele[0]: {ele[1]: i } for ele in j if PurePosixPath(j[0][0]) == PurePosixPath(j[1][0])}


In [14]:
d = {}
for i in range(len(j) - 1):
    d = {ele[0]: {ele[1]: PurePath(ele[0], ele[1], ele[2]) for i in j } for ele in j if PurePosixPath(j[i][0]) == PurePosixPath(j[i+ 1][0])}

In [68]:
sample_input = [
    'CSL/2021/08/24/MS/01/raw_eeg/sub-P001_MS1757_ses-S001_task-Default_run-001_eeg.xdf',
    'CSL/2021/08/24/MS/01/location/smarthelm_gps.csv',
    'CSL/2021/08/26/MS/01/location/smarthelm_gps.csv',
    'CSL/2021/08/26/MS/01/raw_eeg/sub-P001_MS1_ses-S001_task-Default_run-001_eeg.xdf',
]

In [103]:
def get_dict_from_path(sample_input: List[str]) -> Dict[str, Dict[str, str]]:

    parent_path_dict = {}

    child_keys = [
        "raw_eeg",
        "location",
        "vision",
        "attention/",
        "speech/",
        "ui_event"
    ]

        
    def _get_child_key(path):
        for ck in child_keys:
            if ck in ele:
                return ck
    for ele in sample_input:

        child_key = _get_child_key(ele)
        parent_path, child_path = ele.split(child_key)
        # post process the paths - remove leading and trailing "/"
        #parent_path = parent_path.rstrip("/").lstrip("/")
        #child_path = child_path.rstrip("/").lstrip("/")

        if parent_path in parent_path_dict:
            #parent_path_dict[parent_path][child_key] = child_path
            parent_path_dict[parent_path][child_key] = parent_path + child_key + child_path
        else:
            parent_path_dict[parent_path] = {child_key: parent_path + child_key + child_path}
    return parent_path_dict

In [116]:
for k,v in x.items():
    if 'location' in v:
        print(v)

{'attention/': 'CSL/2021/08/26/MS/1/attention/smarthelm_attention.csv', 'location': 'CSL/2021/08/26/MS/1/location/smarthelm_gps.csv', 'raw_eeg': 'CSL/2021/08/26/MS/1/raw_eeg/sub-P001_MS1_ses-S001_task-Default_run-001_eeg.xdf', 'speech/': 'CSL/2021/08/26/MS/1/speech/smarthelm_speech.csv', 'ui_event': 'CSL/2021/08/26/MS/1/ui_event/smarthelm_ui.csv'}
{'attention/': 'CSL/2021/10/01/NP/01/attention/smarthelm_attention.csv', 'location': 'CSL/2021/10/01/NP/01/location/smarthelm_gps.csv', 'raw_eeg': 'CSL/2021/10/01/NP/01/raw_eeg/sub-P001_ses-S001_task-Default_run-001_eeg.xdf', 'speech/': 'CSL/2021/10/01/NP/01/speech/smarthelm_speech.csv', 'ui_event': 'CSL/2021/10/01/NP/01/ui_event/smarthelm_ui.csv', 'vision': 'CSL/2021/10/01/NP/01/vision/GOPR0445.MP4.gz'}
{'attention/': 'CSL/2021/10/01/NP/2/attention/smarthelm_attention.csv', 'location': 'CSL/2021/10/01/NP/2/location/smarthelm_gps.csv', 'raw_eeg': 'CSL/2021/10/01/NP/2/raw_eeg/sub-P001_ses-S001_task-Default_run-001_eeg.xdf', 'speech/': 'CSL/202